# Pre-processamento dos dados

O data set fornecido para este problema não vinha ainda preparado para ser alimentado aos modelos de deeplearning.

Neste notebook são detalhadas todas as estratégias de preprocessamento do das sequências de proteinas.

## Processamento das sequências em subsequências de trimeros

Esta funcão permite apartir de uma sequencia de aminoacidos gerar 3 listas de trimeros

In [ ]:
def subsequence(seq):
    offset0 = re.findall(r'...', seq)
    offset1 = re.findall(r'...', seq[1:])
    offset2 = re.findall(r'...', seq[2:])
    return [offset0, offset1, offset2]


def sequences2subsequences(seq):
    return list(map(subsequence, seq))

## Construção da representação da sequência num vector de 100 dimensões

Esta função ao receber as 3 listas de subsequências e o dicionario de vetores transforma-as em um representação em que os vetores associados a cada trimero são somados num único vetor

In [ ]:
def seq2fixed_length_vec(seqs, vocabulary):
    flat_threemers = [item for sublist in seqs for item in sublist]
    vec = np.array([0.0] * 100)
    for t in flat_threemers:
        try:
            vec = np.add(vec, vocabulary[t])
        except:
            vec = np.add(vec, vocabulary['<unk>'])
    global countline
    countline += 1
    if countline % 10 == 0:
        print(countline)
    print(vec)
    return vec

## Construção da representação com Wordcount/Matriz de vocabulario

Esta função ao receber as 3 listas de subsequências e o dicionario de vetores transforma-as em um representação em que cada posição está associada a um trimero do vocabulario e contem o vetor do mesmo multiplicado pelo número de vezes que aparece na sequência 

In [ ]:
def seq2wordcount_vec(seqs, vocabulary):
    flat_threemers = [item for sublist in seqs for item in sublist]
    wordcount = []
    for val in range(len(vocabulary.keys())):
        wordcount.append([0.0] * 100)
    for t in flat_threemers:
        try:
            wordcount[list(vocabulary.keys()).index(t)] = np.add(wordcount[list(vocabulary.keys()).index(t)],
                                                                 vocabulary[t])
        except:
            wordcount[list(vocabulary.keys()).index('<unk>')] = np.add(
                wordcount[list(vocabulary.keys()).index('<unk>')],
                vocabulary['<unk>'])
    return np.array(wordcount)

## Construção da representação com Matriz e Padding

Esta função ao receber as 3 listas de subsequências e o dicionario de vetores transforma-as em um representação em que cada trimero é substituido pelo vetor correspondente. Seguidamente é efectuado padding com vetores 0 de 100 dimensões até atingir a max_len - 2 de modo a obter uma representação consistente

In [ ]:
def seq2fixed_vec_matrix(seqs, vocabulary, maxlen):
    flat_threemers = [item for sublist in seqs for item in sublist]
    vec_array = []
    for t in flat_threemers:
        try:
            vec_array.append(vocabulary[t])
        except:
            vec_array.append(vocabulary['<unk>'])
    while len(vec_array) < maxlen-2:
        vec_array.append(np.array([0.0] * 100))

    global countline
    countline += 1
    if countline % 10 == 0:
        print(countline)
    return np.array(vec_array, dtype='float32')

##  Tratamento do E.C Number

O E.C numbér é a coluna objectivo deste dataset, no entanto este é composto 4 niveis separados por pontos esta função permite escolher o numero de niveis a considerar bem como eliminar proteinas com 2 EC numbers incompativeis ou sem classificação para o nivel pedido.

In [ ]:
def handle_ec_number(ec_number, ecn_level):
    if ecn_level <= 0 or ecn_level > 4:
        raise ValueError('EC number level must be between 1-4')
    keep = True
    numbers = ec_number.split(';')
    ret = numbers[0].split('.')[:ecn_level]
    for n in numbers:
        n = n.split('.')[:ecn_level]
        if ret != n or '-' in n:
            keep = False
    if keep:
        return ','.join(ret)
    else:
        return None